<a href="https://colab.research.google.com/github/dcc7tk/DS2002/blob/main/project_v6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# packages
import json
import pandas as pd
import json
import requests
import sqlite3
import calendar

In [ ]:
# prompt user to choose cities
while True:
    try:
        cityCount=int(input("How many current weather observations do you want to search? "))
        break
    except ValueError:
        print("Please enter a number.")

# initialize for later
weatherList=[]

# loop for each city given
for i in range(cityCount):
    # prompt/reminder for each city
    locations=input(f"Enter city {i+1}: ")
    api_key="REDACTED"
    url=f"http://api.weatherstack.com/current?access_key={api_key}&query={locations}&air_quality=1"

    response=requests.get(url)

    # if call is successful
    if response.status_code== 200:
        data=response.json()

        # if location is real
        if "location" in data and data["location"]:
            # pull data
            extract={
                # geographical
                "City": data["location"]["name"],
                "Country": data["location"]["country"],
                "Latitude": float(data["location"]["lat"]),
                "Longitude": float(data["location"]["lon"]),
                "Timezone": data["location"]["timezone_id"],
                # timestamp
                "DateTime": data["location"]["localtime"],
                # weather
                "Temperature_C": data["current"]["temperature"],
                "Condition_Text": data["current"]["weather_descriptions"][0],
                "Wind_Speed_kmh": data["current"]["wind_speed"],
                "Wind_Degree": data["current"]["wind_degree"],
                "Wind_Direction": data["current"]["wind_dir"],
                "Pressure_mb": data["current"]["pressure"],
                "Precipitation_mm": data["current"]["precip"],
                "Humidity_pct": data["current"]["humidity"],
                "Cloud_Cover": data["current"]["cloudcover"],
                "Feels_Like_C": data["current"]["feelslike"],
                "Visibility_km": data["current"]["visibility"],
                "UV_Index": data["current"]["uv_index"],
                # air
                "Air_Quality_CO": data["current"]["air_quality"]["co"],
                "Air_Quality_NO2": data["current"]["air_quality"]["no2"],
                "Air_Quality_O3": data["current"]["air_quality"]["o3"],
                "Air_Quality_SO2": data["current"]["air_quality"]["so2"],
                "Air_Quality_PM2_5": data["current"]["air_quality"]["pm2_5"],
                "Air_Quality_PM10": data["current"]["air_quality"]["pm10"],
                "Air_Quality_US_EPA_Index": data["current"]["air_quality"]["us-epa-index"],
                "Air_Quality_GB_DEFRA_Index": data["current"]["air_quality"]["gb-defra-index"],
                # astronomy
                "Sunrise": data["current"]["astro"]["sunrise"],
                "Sunset": data["current"]["astro"]["sunset"],
                "Moonrise": data["current"]["astro"]["moonrise"],
                "Moonset": data["current"]["astro"]["moonset"],
                "Moon_Phase": data["current"]["astro"]["moon_phase"],
                "Moon_Illumination": data["current"]["astro"]["moon_illumination"]
            }

            # store the data
            weatherList.append(extract)
        # if location isn't real
        else:
            print(f"Error: The location '{locations}' could not be found. Please enter a valid city.")
    # if call is unsuccessful
    else:
        print(f"There was an error fetching data for {locations}. Please check the city name or try again later.")

# print
print("\nCollected data:")
for entry in weatherList:
    print(entry)

# put into pandas df
currentDF=pd.DataFrame(weatherList)
print("\nAPI data inserted into pandas dataframe.")

# fix currentDF
currentDF["DateTime"]=pd.to_datetime(currentDF["DateTime"])

# API summary
print("\nSummary of API data:")
print(f"Number of rows: {currentDF.shape[0]}")
print(f"Number of columns: {currentDF.shape[1]}")

How many current weather observations do you want to search? 2
Enter city 1: london
Enter city 2: paris

Collected data:
{'City': 'London', 'Country': 'United Kingdom', 'Latitude': 51.517, 'Longitude': -0.106, 'Timezone': 'Europe/London', 'DateTime': '2025-03-25 03:15', 'Temperature_C': 4, 'Condition_Text': 'Mist', 'Wind_Speed_kmh': 7, 'Wind_Degree': 260, 'Wind_Direction': 'W', 'Pressure_mb': 1021, 'Precipitation_mm': 0, 'Humidity_pct': 100, 'Cloud_Cover': 0, 'Feels_Like_C': 3, 'Visibility_km': 2, 'UV_Index': 0, 'Air_Quality_CO': '527.25', 'Air_Quality_NO2': '87.135', 'Air_Quality_O3': '13', 'Air_Quality_SO2': '8.88', 'Air_Quality_PM2_5': '77.885', 'Air_Quality_PM10': '87.32', 'Air_Quality_US_EPA_Index': '4', 'Air_Quality_GB_DEFRA_Index': '4', 'Sunrise': '05:52 AM', 'Sunset': '06:22 PM', 'Moonrise': '04:38 AM', 'Moonset': '12:55 PM', 'Moon_Phase': 'Waning Crescent', 'Moon_Illumination': 25}
{'City': 'Paris', 'Country': 'France', 'Latitude': 48.867, 'Longitude': 2.333, 'Timezone': 'Euro

In [ ]:
# bring in csv
globalDF=pd.read_csv('/content/GlobalWeatherRepository.csv', sep=',', encoding='latin1')
print("Global weather data inserted into pandas dataframe.")

# initial global summary
print("\nInitial summary of global data:")
print(f"Number of rows: {globalDF.shape[0]}")
print(f"Number of columns: {globalDF.shape[1]}")

# removing made-up observations
globalDF=globalDF[~((globalDF["location_name"]== "Bern") & (globalDF["country"] != "Switzerland"))]
globalDF=globalDF[~((globalDF["location_name"]== "Ivory Ivory Ban"))]
globalDF=globalDF[~((globalDF["location_name"]== "-Kingdom"))]
globalDF=globalDF[~((globalDF["country"]== "Marrocos"))]
globalDF=globalDF[~((globalDF["location_name"]== "Mexico (Grupo Mexico)"))]
globalDF=globalDF[~((globalDF["location_name"]== "Moldova"))]
globalDF=globalDF[~((globalDF["location_name"]== "Krasnyy Turkmenistan"))]
globalDF=globalDF[~((globalDF["location_name"]== "Costa Rica"))]
globalDF=globalDF[~((globalDF["location_name"]== "Lom") & (globalDF["country"]== "Russia"))]

# fixing typos
globalDF.loc[globalDF["location_name"]== "New Guatemala", "country"]="Guatemala"
globalDF.loc[globalDF["location_name"]== "New Guatemala", "location_name"]="Guatemala City"
globalDF.loc[globalDF["location_name"]== "Yaren", "country"]="Nauru"
globalDF.loc[globalDF["location_name"]== "Beirut", "country"]="Lebanon"
globalDF.loc[globalDF["location_name"]== "Tallinn", "country"]="Estonia"
globalDF.loc[globalDF["location_name"]== "New Delhi", "country"]="India"
globalDF.loc[globalDF["location_name"]== "Bishkek", "country"]="Kyrgyzstan"
globalDF.loc[globalDF["location_name"]== "Riga", "country"]="Latvia"
globalDF.loc[globalDF["location_name"]== "Vientiane", "country"]="Laos"
globalDF.loc[globalDF["location_name"]== "Morocco City", "country"]="Morocco"
globalDF.loc[globalDF["location_name"]== "Seoul", "country"]="South Korea"
globalDF.loc[globalDF["location_name"]== "Kingstown", "country"]="Saint Vincent and the Grenadines"
globalDF.loc[globalDF["location_name"]== "Ar Riyadh", "country"]="Saudi Arabia"
globalDF.loc[globalDF["country"]== "USA United States of America", "country"]="United States of America"
globalDF.loc[globalDF["location_name"]== "Addis Abeba", "location_name"]="Addis Ababa"
globalDF.loc[(globalDF['country']== 'Belgium') & (globalDF['location_name'] != 'Brussels'), 'location_name']="'s Gravenjansdijk"
globalDF.loc[globalDF["location_name"]== "Beijing Shi", "location_name"]="Beijing"
globalDF.loc[globalDF["location_name"]== "Bogot", "location_name"]="Bohot"
globalDF.loc[globalDF["location_name"]== "Bras", "location_name"]="Brasilia"
globalDF.loc[globalDF["country"]== "Brunei Darussalam", "country"]="Brunei"
globalDF.loc[globalDF["country"]== "Tonga", "location_name"]="Nuku'alofa"
globalDF.loc[globalDF["location_name"]== "Phnum Penh", "location_name"]="Phnom Penh"
globalDF.loc[globalDF["country"]== "United States of America", "location_name"]="Washington, D.C."
globalDF.loc[globalDF["location_name"]== "Ivory", "ation_name"]="Anosy"
globalDF.loc[globalDF["location_name"]== "Kuwait", "location_name"]="Kuwait City"

# fix globalDF
globalDF=globalDF.rename(columns={'last_updated': 'DateTime'})
globalDF=globalDF.rename(columns={'location_name': 'City'})

# global summary
print("\nSummary of cleaned global data:")
print(f"Number of rows: {globalDF.shape[0]}")
print(f"Number of columns: {globalDF.shape[1]}")

Global weather data inserted into pandas dataframe.

Initial summary of global data:
Number of rows: 60607
Number of columns: 41

Summary of cleaned global data:
Number of rows: 60588
Number of columns: 42


In [ ]:
# rename columns to match currentDF
globalDF=globalDF.rename(columns={
    'country': 'Country',
    'City': 'City',
    'latitude': 'Latitude',
    'longitude': 'Longitude',
    'timezone': 'Timezone',
    'DateTime': 'DateTime',
    'temperature_celsius': 'Temperature_C',
    'condition_text': 'Condition_Text',
    'wind_mph': 'Wind_Speed_kmh',
    'wind_degree': 'Wind_Degree',
    'wind_direction': 'Wind_Direction',
    'pressure_mb': 'Pressure_mb',
    'precip_mm': 'Precipitation_mm',
    'humidity': 'Humidity_pct',
    'cloud': 'Cloud_Cover',
    'feels_like_celsius': 'Feels_Like_C',
    'visibility_km': 'Visibility_km',
    'uv_index': 'UV_Index',
    'air_quality_Carbon_Monoxide': 'Air_Quality_CO',
    'air_quality_Nitrogen_dioxide': 'Air_Quality_NO2',
    'air_quality_Ozone': 'Air_Quality_O3',
    'air_quality_Sulphur_dioxide': 'Air_Quality_SO2',
    'air_quality_PM2.5': 'Air_Quality_PM2_5',
    'air_quality_PM10': 'Air_Quality_PM10',
    'air_quality_us-epa-index': 'Air_Quality_US_EPA_Index',
    'air_quality_gb-defra-index': 'Air_Quality_GB_DEFRA_Index',
    'sunrise': 'Sunrise',
    'sunset': 'Sunset',
    'moonrise': 'Moonrise',
    'moonset': 'Moonset',
    'moon_phase': 'Moon_Phase',
    'moon_illumination': 'Moon_Illumination'
})

# columns to keep
columnsKeep=['City', 'Country', 'Latitude', 'Longitude', 'Timezone', 'DateTime', 'Temperature_C',
    'Condition_Text', 'Wind_Speed_kmh', 'Wind_Degree', 'Wind_Direction', 'Pressure_mb',
    'Precipitation_mm', 'Humidity_pct', 'Cloud_Cover', 'Feels_Like_C', 'Visibility_km',
    'UV_Index', 'Air_Quality_CO', 'Air_Quality_NO2', 'Air_Quality_O3', 'Air_Quality_SO2',
    'Air_Quality_PM2_5', 'Air_Quality_PM10', 'Air_Quality_US_EPA_Index', 'Air_Quality_GB_DEFRA_Index',
    'Sunrise', 'Sunset', 'Moonrise', 'Moonset', 'Moon_Phase', 'Moon_Illumination']

# filter
globalDF=globalDF[columnsKeep]
globalDF['DateTime']=pd.to_datetime(globalDF['DateTime'], format='%m/%d/%Y %H:%M')
filterDF=globalDF.copy()
filterDF=filterDF[filterDF["City"].isin(currentDF["City"])]

# merge
mergeDF=pd.concat([filterDF, currentDF], ignore_index=True)
mergeDF['DateTime']=pd.to_datetime(mergeDF['DateTime'], errors='coerce')

print("\nHistorical and current data about your city/cities inserted into pandas dataframe.")

# merge summary
print("\nSummary of Merged Data:")
print(f"Number of records: {mergeDF.shape[0]}")
print(f"Number of columns: {mergeDF.shape[1]}")



Historical and current data about your city/cities inserted into pandas dataframe.

Summary of Merged Data:
Number of records: 624
Number of columns: 32


In [ ]:
# making tables

# connect to database
connection=sqlite3.connect('weatherDB.sqlite')
print("Database created and connected.")

# cursor
cursor=connection.cursor()
print("Cursor created.")

# global table (aka globalDF)
cursor.execute('''
CREATE TABLE IF NOT EXISTS global (
    Country TEXT,
    City TEXT,
    Latitude REAL,
    Longitude REAL,
    Timezone TEXT,
    DateTime TEXT,
    Temperature_C REAL,
    Condition_Text TEXT,
    Wind_Speed_kmh REAL,
    Wind_Degree INTEGER,
    Wind_Direction TEXT,
    Pressure_mb REAL,
    Precipitation_mm REAL,
    Humidity_pct REAL,
    Cloud_Cover REAL,
    Feels_Like_C REAL,
    Visibility_km REAL,
    UV_Index REAL,
    Air_Quality_CO REAL,
    Air_Quality_O3 REAL,
    Air_Quality_NO2 REAL,
    Air_Quality_SO2 REAL,
    Air_Quality_PM2_5 REAL,
    Air_Quality_PM10 REAL,
    Air_Quality_US_EPA_Index REAL,
    Air_Quality_GB_DEFRA_Index REAL,
    Sunrise TEXT,
    Sunset TEXT,
    Moonrise TEXT,
    Moonset TEXT,
    Moon_Phase TEXT,
    Moon_Illumination REAL
);
''')
print("Global table created.")

# insert globalDF into global table
globalDF.to_sql('global', connection, if_exists='replace', index=False)
print("Data has been inserted into the global table.")

# location table (aka mergeDF)
cursor.execute('''
CREATE TABLE IF NOT EXISTS location (
    Country TEXT,
    City TEXT,
    Latitude REAL,
    Longitude REAL,
    Timezone TEXT,
    DateTime TEXT,
    Temperature_C REAL,
    Condition_Text TEXT,
    Wind_Speed_kmh REAL,
    Wind_Degree INTEGER,
    Wind_Direction TEXT,
    Pressure_mb REAL,
    Precipitation_mm REAL,
    Humidity_pct REAL,
    Cloud_Cover REAL,
    Feels_Like_C REAL,
    Visibility_km REAL,
    UV_Index REAL,
    Air_Quality_CO REAL,
    Air_Quality_O3 REAL,
    Air_Quality_NO2 REAL,
    Air_Quality_SO2 REAL,
    Air_Quality_PM2_5 REAL,
    Air_Quality_PM10 REAL,
    Air_Quality_US_EPA_Index REAL,
    Air_Quality_GB_DEFRA_Index REAL,
    Sunrise TEXT,
    Sunset TEXT,
    Moonrise TEXT,
    Moonset TEXT,
    Moon_Phase TEXT,
    Moon_Illumination REAL
);
''')
print("Location-based table created.")

# insert mergeDF into location table
mergeDF.to_sql('location', connection, if_exists='replace', index=False)
print("Data has been inserted into the location table.")

# close
connection.commit()
connection.close()

Database created and connected.
Cursor created.
Global table created.
Data has been inserted into the global table.
Location-based table created.
Data has been inserted into the location table.


In [ ]:
# query 1: basic exploration
connection=sqlite3.connect('weatherDB.sqlite')
cursor=connection.cursor()

# distinct cities in 'global' table
cursor.execute("SELECT COUNT(DISTINCT City) FROM global;")
result_global=cursor.fetchone()
print(f"Number of cities in altogether: {result_global[0]}")

print("\nAverage temperature for each country altogether:")
# average temperature by country
cursor.execute("""
    SELECT Country, AVG(Temperature_C) AS Avg_Temperature_C
    FROM global
    GROUP BY Country
""")
result_global=cursor.fetchall()
for row in result_global:
    country=row[0]
    avg_temp=round(row[1], 2)  # Round to 2 decimal places
    print(f"{country}: {avg_temp}°C")

# average temperature for selected city/cities
cursor.execute("""
    SELECT City, AVG(Temperature_C) AS Avg_Temperature_C
    FROM location
    GROUP BY City
""")
result_location=cursor.fetchall()
print("\nAverage temperature for each selected city:")
for row in result_location:
    city=row[0]
    avg_temp=round(row[1], 2)  # Round to 2 decimal places
    print(f"{city}: {avg_temp}°C")

# close
connection.close()

Number of cities in altogether: 229

Average temperature for each country altogether:
Afghanistan: 18.15°C
Albania: 22.05°C
Algeria: 22.73°C
Andorra: 10.99°C
Angola: 27.14°C
Antigua and Barbuda: 27.0°C
Argentina: 15.21°C
Armenia: 18.28°C
Australia: 12.58°C
Austria: 14.57°C
Azerbaijan: 18.79°C
Bahamas: 25.11°C
Bahrain: 29.61°C
Bangladesh: 30.0°C
Barbados: 27.54°C
Belarus: 12.04°C
Belgium: 12.96°C
Belize: 23.0°C
Benin: 29.6°C
Bhutan: 11.48°C
Bolivia: 17.96°C
Bosnia and Herzegovina: 15.84°C
Botswana: 27.26°C
Brazil: 25.72°C
Brunei: 28.05°C
Bulgaria: 17.66°C
Burkina Faso: 30.44°C
Burundi: 27.41°C
Cambodia: 30.53°C
Cameroon: 26.12°C
Canada: 5.84°C
Cape Verde: 25.41°C
Central African Republic: 31.13°C
Chad: 31.68°C
Chile: 11.02°C
China: 15.79°C
Colombia: 13.5°C
Comoros: 26.18°C
Congo: 29.26°C
Costa Rica: 18.72°C
Croatia: 16.35°C
Cuba: 23.65°C
Cyprus: 25.82°C
Czech Republic: 12.72°C
Democratic Republic of Congo: 29.15°C
Denmark: 11.95°C
Djibouti: 32.04°C
Dominica: 27.11°C
Dominican Republic: 

In [ ]:
# query 2: air quality measures for selected city/cities
connection=sqlite3.connect('weatherDB.sqlite')
cursor=connection.cursor()

# query
query2="""
    SELECT City,
           ROUND(AVG(Air_Quality_CO), 2) AS Average_CO,
           ROUND(AVG(Air_Quality_NO2), 2) AS Average_NO2,
           ROUND(AVG(Air_Quality_O3), 2) AS Average_O3,
           ROUND(AVG(Air_Quality_SO2), 2) AS Average_SO2,
           ROUND(AVG(Air_Quality_US_EPA_Index), 2) AS Average_EPA_Index
    FROM location
    GROUP BY City;
"""

# execute
cursor.execute(query2)
results=cursor.fetchall()

# create a df
columns2=['City', 'Average_CO', 'Average_NO2', 'Average_O3', 'Average_SO2', 'Average_EPA_Index']
df2=pd.DataFrame(results, columns=columns2)
print(df2)

# close
connection.close()

     City  Average_CO  Average_NO2  Average_O3  Average_SO2  Average_EPA_Index
0  London      351.99        40.10       55.14         8.39               1.53
1   Paris      460.78        42.67       51.90         6.42               1.72


In [ ]:
# query 3pt1: top 10 cities with the highest summer UV index
connection=sqlite3.connect('weatherDB.sqlite')
cursor=connection.cursor()

query3pt1="""
  SELECT City, Country,
    AVG(UV_Index) AS Avg_UV_Index
  FROM global
  WHERE strftime('%m', DateTime) IN ('06', '07', '08')
  GROUP BY City, Country
  ORDER BY Avg_UV_Index DESC
  LIMIT 10;
"""
cursor.execute(query3pt1)
results=cursor.fetchall()

print("Highest average summer UV indices:")
# print as a list
for index, row in enumerate(results, start=1):
    print(f"{index}. {row[0]}, {row[1]}'s average summer UV index: {round(row[2], 2)}")

# Close the connection
connection.close()

Highest average summer UV indices:
1. Ar Riyadh, Saudi Arabia's average summer UV index: 11.0
2. Baghdad, Iraq's average summer UV index: 10.48
3. Riyadh, Saudi Arabia's average summer UV index: 10.08
4. Kuwait City, Kuwait's average summer UV index: 9.8
5. Kinshasa, Democratic Republic of Congo's average summer UV index: 9.61
6. Brazzaville, Congo's average summer UV index: 9.6
7. Cairo, Egypt's average summer UV index: 9.33
8. Doha, Qatar's average summer UV index: 9.31
9. Damascus, Syria's average summer UV index: 8.67
10. Ashgabat, Turkmenistan's average summer UV index: 8.57


In [ ]:
# query 4: full moon dates
connection=sqlite3.connect('weatherDB.sqlite')
cursor=connection.cursor()

query4="""
  SELECT City, Country, DateTime, Moon_Phase
  FROM location
  WHERE Moon_Phase='Full Moon'
  ORDER BY City, DateTime DESC
"""
cursor.execute(query4)
results=cursor.fetchall()

# collect the most recent
recentFM={}
for row in results:
    city, country, datetime, moon_phase=row
    if city not in recentFM:
        recentFM[city]=(country, datetime, moon_phase)

print("Most recent full moon for each city:")
for city, info in recentFM.items():
    country, datetime, moon_phase=info
    print(f"{city}, {country}: {datetime}")

# close
connection.close()

Most recent full moon for each city:
London, United Kingdom: 2025-03-14 09:45:00
Paris, France: 2025-03-14 10:45:00
